<a href="https://colab.research.google.com/github/COMM599-Fall2023/fall_2023/blob/yueqihua-final-project/student%20folders/yueqihua/final/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection

---

*Using twitch_data_full.csv to create a cleaned Twitch dataset and a games dataset with game information scraped from Steam*

*   Input: twitch_data_full.csv
*   Output: twitch_data.csv, game_data.csv

## Install dependencies, import modules, input credentials

In [ ]:
!pip install pandas

In [ ]:
from getpass import getpass
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re
import time
from tqdm import tqdm
import unicodedata

## Twitch Streaming Data

Using a [dataset from Kaggle](https://www.kaggle.com/datasets/rankirsh/evolution-of-top-games-on-twitch)

In [ ]:
# loading and cleaning dataset

twitch_df = pd.read_csv("datasets/twitch_data_full.csv", encoding='latin-1', index_col=0)

twitch_df = twitch_df.reset_index(drop=False)
twitch_df.columns = twitch_df.columns.str.lower()
twitch_df['game'] = twitch_df['game'].apply(lambda game: unicodedata.normalize('NFC', str(game))) # converting unicode into special characters
twitch_df['month'] = twitch_df['month'].apply(lambda x: f"{x:02d}") # formatting months from "3" to "03"
twitch_df['year'] = twitch_df['year'].astype(str)

twitch_df.head()

,rank,game,month,year,hours_watched,hours_streamed,peak_viewers,peak_channels,streamers,avg_viewers,avg_channels,avg_viewer_ratio
0,1,League of Legends,01,2016,94377226,1362044,530270,2903,129172,127021,1833,69.29
1,2,Counter-Strike: Global Offensive,01,2016,47832863,830105,372654,2197,120849,64378,1117,57.62
2,3,Dota 2,01,2016,45185893,433397,315083,1100,44074,60815,583,104.26
3,4,Hearthstone,01,2016,39936159,235903,131357,517,36170,53749,317,169.29
4,5,Call of Duty: Black Ops III,01,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03


In [ ]:
twitch_df.dtypes

rank                  int64
game                 object
month                object
year                 object
hours_watched         int64
hours_streamed        int64
peak_viewers          int64
peak_channels         int64
streamers             int64
avg_viewers           int64
avg_channels          int64
avg_viewer_ratio    float64
dtype: object

In [ ]:
# checking how many rows of data each game has

game_count = twitch_df.groupby('game')['game'].count()
game_count = game_count.sort_values(ascending=False)

game_count_df = pd.DataFrame({'count': game_count})
game_count_df.reset_index(inplace=True)

game_count_df # 2150 x 2

,game,count
0,Dungeons & Dragons,94
1,Retro,94
2,Magic: The Gathering,93
3,Warcraft III,93
4,Super Mario 64,93
...,...,...
2145,Left 4 Dead,1
2146,Last Tide,1
2147,LEGO Worlds,1
2148,LEGO The Lord of the Rings,1


## Steam Games Data

Using Twitch dataset to get Steam game data

##### Filtering out games from Twitch dataset that are not available on Steam

In [ ]:
# defining functions

def get_steam_game_list(): # gets all available apps on steam (includes dlcs, skin packs, etc)
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2"
    params = {
    }

    try:
        response = requests.get(url, params=params, timeout=15)
        all_steam_games = response.json()['applist']['apps']
        all_steam_games = [remove_registered_symbol(game.copy()) for game in all_steam_games]
        return all_steam_games
    except:
        print("Error fetching the Steam game list.")
        return []

def remove_registered_symbol(game): # removes unnecessary symbols
    game['name'] = re.sub(r'®', '', game['name'])
    return game

def check_games_on_steam(twitch_games, steam_games): # compares games in the twitch dataset and the steam website
    steam_game_names = {game['name']: game['appid'] for game in steam_games}
    available_on_steam = {}

    to_omit = ['Retro', 'Dofus', 'Just Chatting', 'Live'] # these categories are not game-related but are names of games on steam

    for game in twitch_games:
        if game in steam_game_names and game not in to_omit:
            appid = steam_game_names[game]
            available_on_steam[str(appid)] = game

    return available_on_steam

In [ ]:
# getting list of games in the twitch dataset

twitch_games = twitch_df['game'].drop_duplicates().tolist()
print(len(twitch_games)) # 2150

2150


In [ ]:
#  getting list of all games on steam

all_steam_games = get_steam_game_list()
print(len(all_steam_games))

181671


In [ ]:
# finding common games in twitch_games and all_steam_games

game_list = check_games_on_steam(twitch_games, all_steam_games)
print(len(game_list)) # 1120 is what i got when i ran it but sometimes it returns different numbers depending on the API

1120


##### Scraping Steam game information from game_list

In [ ]:
### optional: uncomment and load dataset instead of scraping, then you can skip the scraping part
# game_data_df = pd.read_csv("datasets/game_data.csv", encoding='utf-8-sig', index_col=0)

In [ ]:
# defining functions

def get_game_data(appid, game) : # returns game data for an appid
    url = "https://store.steampowered.com/api/appdetails"
    params = {
        'appids' : appid,
        'cc' : 'US',
        'l' : 'en'
    }

    try :
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()

        result = response.json()
        if appid not in result or "data" not in result[appid] :
            print("Data not found for game: ", game)
            return None

        data = result[appid]['data']

        input_format = "%b %d, %Y"
        output_format = "%Y-%m-%d"
        date_string = data['release_date']['date']
        date_input = datetime.strptime(date_string, input_format)
        release_date = date_input.strftime(output_format)

        description = BeautifulSoup(data['detailed_description'], 'html.parser')
        description = description.get_text()

        if data['is_free'] :
            price = 0
        else :
            price = data['price_overview']['final'] / 100 # changing units from cents to dollars

        platforms_list = [ key.capitalize() for key, value in data['platforms'].items() if value ]
        platforms = ", ".join(platforms_list)

        genres_list = [ genre["description"] for genre in data['genres'] ]
        genres = ", ".join(genres_list)

        developers = ", ".join(data['developers'])
        publishers =  ", ".join(data['publishers'])

        if 'metacritic' in data and 'score' in data['metacritic']:
            score = data['metacritic']['score']
        else:
            score = None

        game_data = {
            'appid' : data['steam_appid'],
            'game' : game,
            'release_date' : release_date,
            'required_age' : data['required_age'],
            'is_free' : data['is_free'],
            'price' : price,
            'description' : description,
            'developers' : developers,
            'publishers' : publishers,
            'platforms' : platforms,
            'score' : score,
            'genres' : genres
        }

        return game_data
    except :
        print("API request failed for game: ", game)
        time.sleep(30)
        return None

In [ ]:
# testing

get_game_data('322330', 'Don\'t Starve Together')

{'appid': 322330,
 'game': "Don't Starve Together",
 'release_date': '2016-04-21',
 'required_age': 0,
 'is_free': False,
 'price': 5.09,
 'description': 'Explore TogetherDiscover and explore a massive procedurally generated and biome-rich world with countless resources and threats. Whether you stick to the surface world, go spelunking in the caves, dive deeper into the Ancient Archive, or set sail for the Lunar islands, it will be a long time before you run out of things to do.Fight TogetherSeasonal bosses, wandering menaces, lurking shadow creatures, and plenty of flora and fauna ready to turn you into a spooky ghost.Farm TogetherPlow fields and sow seeds to grow the farm of your dreams. Tend to your crops to help your fellow survivors stay fed and ready for the challenges to come.Build TogetherProtect yourself, your friends, and everything you have managed to gather, because you can be sure, somebody or something is going to want it back.About the GameFight, Farm, Build and Explore 

In [ ]:
# this takes around 1 hour

game_data_list = []

count = 0
for appid, game in tqdm(game_list.items(), desc="Processing games") : # iterating through all games in the game list and getting their data
    game_data = get_game_data(appid, game)

    if game_data is None :
        continue
    else :
        game_data_list.append(game_data)

        count += 1
        if count % 50 == 0 :
            print(f"Scraped {count} games. Sleeping for 60 seconds...")
            time.sleep(60) # sleeping because if not the API blocks the requests

game_data_df = pd.DataFrame(game_data_list)

game_data_df # 1076

Processing games:   0%|          | 1/1120 [00:00<05:33,  3.36it/s]

Data not found for game:  League of Legends


Processing games:   1%|          | 7/1120 [00:02<07:52,  2.36it/s]

API request failed for game:  Grand Theft Auto V


Processing games:   1%|          | 12/1120 [01:04<1:27:59,  4.77s/it]

...

Scraped 1000 games. Sleeping for 60 seconds...


Processing games:  97%|█████████▋| 1083/1120 [1:16:45<02:53,  4.68s/it]

API request failed for game:  Blue Protocol


Processing games: 100%|██████████| 1120/1120 [1:17:59<00:00,  4.18s/it]


,appid,game,release_date,required_age,is_free,price,description,developers,publishers,platforms,score,genres
0,570,Dota 2,2013-07-09,0,True,0.00,"The most-played game on Steam.Every day, milli...",Valve,Valve,"Windows, Mac, Linux",90.0,"Action, Strategy, Free to Play"
1,311210,Call of Duty: Black Ops III,2015-11-05,17,False,19.79,Zombies Chronicles Deluxe EditionNow with more...,"Treyarch, Aspyr (Mac)","Activision, Aspyr (Mac)","Windows, Mac",73.0,"Action, Adventure"
2,1407200,World of Tanks,2021-04-28,0,True,0.00,Command over 600 machines from World War II th...,Wargaming Group Limited,Wargaming Group Limited,Windows,80.0,"Action, Massively Multiplayer, Simulation, Fre..."
3,359550,Tom Clancy's Rainbow Six Siege,2015-12-01,17,False,19.99,Edition ComparisonDeluxe EditionThe Tom Clancy...,Ubisoft Montreal,Ubisoft,Windows,NaN,Action
4,1343400,RuneScape,2020-10-14,0,True,0.00,Journey into the Sixth Age of Gielinor and dis...,Jagex Ltd,Jagex Ltd,"Windows, Mac",NaN,"Free to Play, Massively Multiplayer, RPG"
...,...,...,...,...,...,...,...,...,...,...,...,...
1035,2203070,Dragonheir: Silent Gods,2023-10-26,0,True,0.00,Join Our DiscordBug Report1. Join the official...,SGRA Studio,Nuverse Pte. Ltd.,Windows,NaN,"Adventure, Massively Multiplayer, RPG, Strateg..."
1036,2107670,Warhaven,2023-09-20,0,True,0.00,JOIN OUR DISCORD!About the GameENTER THE FRAYW...,NEXON,NEXON,Windows,NaN,"Action, Free to Play, Early Access"
1037,2140510,Town of Salem 2,2023-08-25,0,True,0.00,The Town of Salem is in turmoil as the Coven h...,BlankMediaGames LLC,BlankMediaGames LLC,"Windows, Mac",NaN,"Casual, Indie, RPG, Strategy, Free to Play"
1038,2137700,I'm on Observation Duty 6,2023-09-01,0,False,11.99,The most ambitious I'm on Observation Duty gam...,"Notovia, Dreamloop Games",Notovia,"Windows, Mac, Linux",NaN,Indie


In [ ]:
# basic statistics

print("Size of the dataset:", game_data_df.shape)
print('----------')

max_score = game_data_df[game_data_df['score'] == game_data_df['score'].max()]
min_score = game_data_df[game_data_df['score'] == game_data_df['score'].min()]
print("*** Game Ratings:",
      f"\n\tAverage score: {game_data_df['score'].mean()}"
      f"\n\tHighest scored game: {max_score['game'].values[0]} [{max_score['score'].values[0]}]",
      f"\n\tLowest scored game: {min_score['game'].values[0]} [{min_score['score'].values[0]}]")
print('----------')

print("*** Free or Paid:")
print(f"\tFree: {len(game_data_df[game_data_df['is_free'] == True])}")
print(f"\tPaid: {len(game_data_df[game_data_df['is_free'] == False])}")
print('----------')

print("*** Top:")
for mode in game_data_df['developers'].mode() :
    mode_count = game_data_df['developers'].value_counts()[mode]
    print(f"\tDeveloper: {mode} [{mode_count}]")
for mode in game_data_df['publishers'].mode() :
    mode_count = game_data_df['publishers'].value_counts()[mode]
    print(f"\tPublisher: {mode} [{mode_count}]")

Size of the dataset: (1040, 12)
----------
*** Game Ratings: 
	Average score: 78.87550200803213
	Highest scored game: Disco Elysium [97.0] 
	Lowest scored game: Hello Neighbor [38.0]
----------
*** Free or Paid:
	Free: 118
	Paid: 922
----------
*** Top:
	Developer: CAPCOM Co., Ltd. [15]
	Publisher: Bethesda Softworks [25]


In [ ]:
# save to csv

game_data_df.to_csv("/content/datasets/game_data.csv", index=True, encoding='utf-8-sig')

## Back to Twitch Streaming Data
Filtering Twitch dataset so it only contains games in Steam dataset

In [ ]:
game_list = game_data_df['game'].tolist()

twitch_df.drop(twitch_df[~twitch_df['game'].isin(game_list)].index, inplace=True)
twitch_df.reset_index(drop=True, inplace=True)

twitch_df # 8071 x 12

,rank,game,month,year,hours_watched,hours_streamed,peak_viewers,peak_channels,streamers,avg_viewers,avg_channels,avg_viewer_ratio
0,3,Dota 2,01,2016,45185893,433397,315083,1100,44074,60815,583,104.26
1,5,Call of Duty: Black Ops III,01,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03
2,15,World of Tanks,01,2016,4993627,128356,39358,368,16121,6720,172,38.90
3,16,Tom Clancy's Rainbow Six Siege,01,2016,4866039,242134,25742,694,42244,6549,325,20.10
4,17,RuneScape,01,2016,4632857,74628,18809,167,7628,6235,100,62.08
...,...,...,...,...,...,...,...,...,...,...,...,...
8066,190,I'm on Observation Duty 6,09,2023,581068,4730,24948,30,1249,808,6,122.85
8067,191,Deceit 2,09,2023,575900,3089,43348,55,987,800,4,186.44
8068,194,Valheim,09,2023,568622,75546,8861,203,7975,790,105,7.53
8069,196,PlateUp!,09,2023,560064,18617,16476,71,4034,778,25,30.08


In [ ]:
# games with the most months in the top 200

game_count = twitch_df.groupby('game')['game'].count()
game_count = game_count.sort_values(ascending=False)

game_count_df = pd.DataFrame({'count': game_count})
game_count_df.reset_index(inplace=True)

game_count_df

,game,count
0,Tom Clancy's Rainbow Six Siege,93
1,Dota 2,93
2,Rust,93
3,Path of Exile,93
4,World of Warships,93
...,...,...
1035,LEGO Worlds,1
1036,Last Tide,1
1037,Left 4 Dead,1
1038,Life is Strange: True Colors,1


In [ ]:
# save to csv

twitch_df.to_csv("/content/datasets/twitch_data.csv", index=True, encoding='utf-8-sig')